# Sentiment analysis model and traning with SVC model

1. 介紹4種情感模型使用方式(Baidu, Keymoji, Vader, Google)
2. SVC 訓練為自有模型

In [1]:
import pandas as pd
import numpy as np
import spacy
import pickle
import math
import matplotlib.pyplot as plt
import time
import json

from json import JSONDecodeError
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.preprocessing import minmax_scale

# Load Data

In [ ]:
!gdown --"https://drive.google.com/file/d/1CFn5gFSVBUk3c6DWAebjjhHWxzb_9VOQ/view?usp=sharing"

In [ ]:
Cynes = pd.read_csv("sectorDF.csv").drop("Unnamed: 0", axis = 1)

# Sentiment analysis API
1. (中) Baidu from: https://ai.baidu.com/tech/nlp_apply/sentiment_classify
2. (中) Keymoji from: https://api.droidtown.co/document/#KeyMoji_2
3. (英) VAder from: https://github.com/cjhutto/vaderSentiment
4. (英) Google from: https://cloud.google.com/natural-language

## 1. Baidu 

In [ ]:
APP_ID = 'your APP_ID'
API_KEY = 'your API_KEY'
SECRET_KEY = 'your SECRET_KEY'

### A. Baidu Example

In [ ]:
# !pip install baidu-aip

In [ ]:
from aip import AipNlp            

client = AipNlp(appId = APP_ID, apiKey=API_KEY, secretKey=SECRET_KEY)
text = "在大數據情感分析的幫助下，能夠準確的預測股價的走勢，成功賺大錢" 
client.sentimentClassify(text.encode('gb18030','ignore').decode('gbk','ignore'))

### B. Cynes with Baidu

雖然官方文檔寫最大輸入字數為2048字，但有時輸入超過1065字就爆了，有時965...每次測試不一定。因此以下為若出現 error msg，則砍掉部分字數重跑一次 API 模型

In [ ]:
'''
將出現錯誤訊息的'error_code', 'error_msg' 都重跑
'''
baiduLIST = []
for number in range(len(Cynes["Content"])):
    try:
        time.sleep(0.5)
        text = Cynes["Content"][number]      
        baidu_score = client.sentimentClassify(text.encode('gb18030','ignore').decode('gbk','ignore'))

        if 'error_code' in baidu_score:
            time.sleep(0.5)
            text = Cynes["Content"][number][:1065]     ##縮短文本，重跑第一次
            baidu_score = client.sentimentClassify(text.encode('gb18030','ignore').decode('gbk','ignore'))
            
            if 'error_code' in baidu_score:
                time.sleep(0.5)
                text = Cynes["Content"][number][:965]     ##縮短文本，重跑第二次
                baidu_score = client.sentimentClassify(text.encode('gb18030','ignore').decode('gbk','ignore'))
            
                if 'error_code' in baidu_score:  
                    time.sleep(0.5)
                    text = Cynes["Content"][number][:900]
                    baidu_score = client.sentimentClassify(text.encode('gb18030','ignore').decode('gbk','ignore'))
                    
                    if 'error_code' in baidu_score:               ##縮短文本，重跑第三次
                        time.sleep(0.5)
                        text = Cynes["Content"][number][:850]
                        baidu_score = client.sentimentClassify(text.encode('gb18030','ignore').decode('gbk','ignore'))
                        baiduLIST.append(baidu_score)
                        print(number, "----error_code!!!? try again")      ##縮短文本，考慮重跑第四次
                    else:
                        baiduLIST.append(baidu_score)
                        print(number)
                else:
                        baiduLIST.append(baidu_score)
                        print(number)
            else:    
                baiduLIST.append(baidu_score)
                print(number)
            
        elif 'error_msg' in baidu_score:
            time.sleep(0.5)
            baidu_score = client.sentimentClassify(text.encode('gb18030','ignore').decode('gbk','ignore'))
            baiduLIST.append(baidu_score)
            print(number, "----error_msg檢查看看還有事嗎")
        else:
            baiduLIST.append(baidu_score)
            print(number)
            
    except JSONDecodeError:
        text = Cynes["Content"][number][:1065]
        time.sleep(2)
        baidu_score = client.sentimentClassify(text.encode('gb18030','ignore').decode('gbk','ignore'))
        
        if 'error_code' in baidu_score:
            time.sleep(2)
            text = Cynes["Content"][number][:965]
            baidu_score = client.sentimentClassify(text.encode('gb18030','ignore').decode('gbk','ignore'))
            baiduLIST.append(baidu_score)
            print(number, "----JSONDecodeError檢查看看還有事嗎")
        else:    
            baiduLIST.append(baidu_score)
            print(number)
        
print("done")

In [ ]:
POS_prob = []
NEG_prob = []
Confidence = []
Sentiment = []
try:
    for result_index in range(len(baiduLIST)):
        POS_prob.append(baiduLIST[result_index]['items'][0]['positive_prob'])
        NEG_prob.append(baiduLIST[result_index]['items'][0]['negative_prob'])
        Confidence.append(baiduLIST[result_index]['items'][0]['confidence'])
        Sentiment.append(baiduLIST[result_index]['items'][0]['sentiment'])
except KeyError:
    print(result_index)

In [ ]:
Cynes["POS_prob"] = POS_prob
Cynes["NEG_prob"] = NEG_prob
Cynes["Confidence"] = Confidence
Cynes["Sentiment"] = Sentiment

In [ ]:
Cynes.to_csv()

## 2. Keymoji

### A. KeyMoji Example

In [ ]:
# !pip install KeyMojiAPI

In [ ]:
from KeyMojiAPI import KeyMoji

keymoji = KeyMoji(username="你的username", 
                  keymojiKey="你的keymojiKey")

In [7]:
def KeyMoji_sentiments_score(inputSTR):
    result = keymoji.sense2(inputSTR)
    if result["status"] == True:
        return result['results']
    else:
        print(result["msg"])
        return None

In [11]:
inputSTR = "在大數據情感分析的幫助下，能夠準確的預測股價的走勢，成功賺大錢" 

keymoji.sense2(inputSTR)

{'status': True,
 'msg': 'Success!',
 'results': [{'score': 0.5544,
   'sentiment': 'positive',
   'input_str': '在大數據情感分析的幫助下',
   'cursing': False},
  {'score': 0.4886,
   'sentiment': 'positive',
   'input_str': '能夠準確的預測股價的走勢',
   'cursing': False},
  {'score': 0.6929,
   'sentiment': 'positive',
   'input_str': '成功賺大錢',
   'cursing': False}],
 'sense': 'sense2',
 'version': 'v102'}

### B. Cynes with KeyMoji

In [ ]:
Keymoji_resultLIST = []

for number in range(len(Cynes["Content"])):
    time.sleep(0.5)
    try:
        sentiments_score = KeyMoji_sentiments_score(Cynes["Content"][number])
        Keymoji_resultLIST.append(sentiments_score)
        print(number)
    except TimeoutError:
        time.sleep(3)
        sentiments_score = KeyMoji_sentiments_score(Cynes["Content"][number])
        Keymoji_resultLIST.append(sentiments_score)
        print(number,"ok TimeoutError")
print("done")

In [ ]:
article_scoreLIST = []
for article_index in range(len(Keymoji_resultLIST)):
    sentence_scoreLIST = []
    for sentence_index in range(len(Keymoji_resultLIST[article_index])):
        sentence_score = Keymoji_resultLIST[article_index][sentence_index]["score"]
        sentence_scoreLIST.append(sentence_score)
    article_score = mean(sentence_scoreLIST)
    article_scoreLIST.append(article_score)
    
Cynes["Keymoji_score"] = [round(score,2) for score in article_scoreLIST]

In [ ]:
Cynes.to_csv()

## 3.VAder 

### A. VAder Example

In [ ]:
# !pip install vaderSentiment

In [15]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [18]:
analyzer = SentimentIntensityAnalyzer()

analyzer.polarity_scores("The phone is super cool.")

{'neg': 0.0, 'neu': 0.326, 'pos': 0.674, 'compound': 0.7351}

### B. Cynes with VAder

In [ ]:
Vader_resultLIST = []
for number in range(len(Cynes["Content"])):
    score = analyzer.polarity_scores(Cynes["Content"][number])
    Vader_resultLIST.append(score)
print("done")

In [ ]:
Vader_SentimentDF = pd.DataFrame(Vader_resultLIST)
Vader_SentimentDF

In [ ]:
Cynes = pd.merge(Cynes, Vader_SentimentDF,left_index=True, right_index=True)

In [ ]:
Cynes.to_csv()

## 4. Google

### A. Google Example

In [ ]:
# !pip install --upgrade google-cloud-language

In [ ]:
from google.cloud import language_v1
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "your json file"

In [ ]:
client = language_v1.LanguageServiceClient()
type_ = language_v1.Document.Type.PLAIN_TEXT 

In [ ]:
client = language_v1.LanguageServiceClient()

def extract_entities(text):
    document = language_v1.Document(
        content=text,
        type_="PLAIN_TEXT",
    )
    res_object = client.analyze_entity_sentiment(
        document=document, 
        encoding_type="UTF32",
    )
    return res_object.entities

In [ ]:
text = """
  David P. Bauer, President and Chief Executive Officer of National Fuel Gas Company, stated: “National Fuel had an outstanding start to fiscal 2022, with adjusted operating results increasing 40% from the prior year, primarily driven by strong performance from our Exploration & Production business. We continued to execute on our growth plans by placing the FM100 expansion and modernization project into service in December. This project, which was completed on-time and under budget, provides significant new capacity on our FERC-regulated pipeline system which, along with Transco’s companion Leidy South project, is an important, long-term valuable outlet for the Company’s Appalachian production. Further, the project incorporated best-in-class emissions controls, including the installation of vent gas recovery systems and compressed air pneumatics, maintaining our focus on reducing the methane intensity of our operations.
“Additionally, throughout the quarter, National Fuel continued to make significant progress on its sustainability initiatives, achieving certification of 100% of our Appalachian production under Equitable Origin’s EO100TM Standard for Responsible Energy Development. This accreditation along with our ongoing investments to achieve our emission reduction targets, positions National Fuel to differentiate its responsibly sourced production in the marketplace, including all of our volumes transported on the Leidy South project.
“As we look forward, National Fuel’s integrated businesses are well-positioned for the future. Our unique mix of assets and the strength of our balance sheet enable us to deliver growth, while enhancing our ability to generate long-term free cash flow.”
"""

for ent in extract_entities(text):
    print(f"Name: {ent.name}")
    print(f"Sentiment Score: {ent.sentiment.score}")
    print(f"Sentiment Magnitude(強度): {ent.sentiment.magnitude}")
    print("="*10)

### B. Cynes with Google

In [ ]:
scoreLIST = []
magnitudeLIST =[]

for i in range(len(Cynes["Content"])):
    time.sleep(0.5)
    text = Cynes["Content"].iloc[i]
    
    language = "en"
    document = {"content": text, "type_": type_, "language": language}
    encoding_type = language_v1.EncodingType.UTF8
    response = client.analyze_sentiment(request = {'document': document, 'encoding_type': encoding_type})
    
    sentiment_score = response.document_sentiment.score
    sentiment_magnitude = response.document_sentiment.magnitude
    
    scoreLIST.append(sentiment_score)
    magnitudeLIST.append(sentiment_magnitude)
    
    print(i,"ok")

In [ ]:
Cynes["Google_score"] = scoreLIST
Cynes["Google_magnitude"] = magnitudeLIST

In [ ]:
Cynes.to_csv()

# Spacy Embedding

## A. Spacy Example

In [ ]:
# conda install -c conda-forge spacy

In [ ]:
# !python -m spacy download zh_core_web_md

In [ ]:
nlp = spacy.load('zh_core_web_md')
text = "隨著北約各國逐漸將重型武器送往烏克蘭支援反攻，德國在送出豹一戰車"
nlp(text).vector

## B. Cynes embeding by Spacy

In [ ]:
nlp = spacy.load('zh_core_web_md')
train_x_vectors = [nlp(text_).vector for text_ in Cynes['Content']]

In [ ]:
Cynes["Spacy_vector"] = train_x_vectors

# SVC Modelling

以 Baidu 為例

In [ ]:
condlist = [Cynes["Sentiment"] == 0,
            Cynes["Sentiment"] == 1, 
            Cynes["Sentiment"] == 2]
choicelist = ["neg", "neu", "pos"]
Cynes["Sentiment_trans"] = np.select(condlist, choicelist)
Cynes

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(list(Cynes["Spacy_vector"]), Cynes["Sentiment_trans"], test_size=0.2, random_state=66)
clf_svm = svm.SVC(kernel='linear', probability=True)  ## probability要打開 後續才能有預測機率序列
clf_svm.fit(X_train, y_train)

In [ ]:
accuracy_score(y_true = y_train,
               y_pred = clf_svm.predict(X_train))

In [ ]:
accuracy_score(y_true = y_test,
               y_pred = clf_svm.predict(X_test))

In [ ]:
##預測結果
clf_svm.predict(X_test)

In [ ]:
##實際結果
y_test

In [ ]:
##預測機率
clf_svm.predict_proba(X_test)

In [ ]:
print(classification_report(y_true = y_test,
                            y_pred = clf_svm.predict(X_test)))

In [ ]:
# save the model to disk
filename = 'svc_model.sav'
pickle.dump(model, open(filename, 'wb'))

In [ ]:
# # load the model from disk

# loaded_model = pickle.load(open(filename, 'rb'))
# result = loaded_model.score(X_test, Y_test)